In [4]:
import heapq

def heuristic(state, goal):
    # Manhattan distance for each tile
    return sum(abs(s % 3 - g % 3) + abs(s // 3 - g // 3)
               for s, g in zip(state, goal) if s != 0)

def get_neighbors(state):
    neighbors = []
    idx = state.index(0) # Find the index of the empty spot (0)
    moves = []

    # Check possible moves (up, down, left, right)
    if idx not in [0, 1, 2]: moves.append(-3) # Up
    if idx not in [6, 7, 8]: moves.append(3)  # Down
    if idx not in [0, 3, 6]: moves.append(-1) # Left
    if idx not in [2, 5, 8]: moves.append(1)  # Right

    for move in moves:
        new_state = state[:]
        # Swap the empty spot with the tile
        new_state[idx], new_state[idx + move] = new_state[idx + move], new_state[idx]
        neighbors.append(new_state)
    return neighbors

def a_star_8_puzzle(start, goal):
    # open_list stores (f_cost, g_cost, state, path)
    open_list = []
    # (f, g, state, path)
    heapq.heappush(open_list, (heuristic(start, goal), 0, start, []))

    visited = set() # Stores states we've already processed

    while open_list:
        f, g, current, path = heapq.heappop(open_list)

        # --- Goal Check ---
        if current == goal:
            return path + [current] # Return the complete path

        # --- Visited Check ---
        # If we've seen this state before, skip it
        if tuple(current) in visited:
            continue
        visited.add(tuple(current)) # Mark as visited

        # --- Explore Neighbors ---
        for neighbor in get_neighbors(current):
            if tuple(neighbor) not in visited:
                new_g = g + 1
                new_h = heuristic(neighbor, goal)
                new_f = new_g + new_h
                heapq.heappush(open_list, (new_f, new_g, neighbor, path + [current]))

    return None # No solution found

# --- VISUALIZATION FUNCTION ---
def visualize_puzzle(state):
    """Prints a 3x3 grid for a given puzzle state."""
    print("-------")
    for i in range(3):
        row = state[i*3 : i*3 + 3]
        # Format the row, replacing 0 with a blank space
        print(f"| {row[0] if row[0] != 0 else ' '} {row[1] if row[1] != 0 else ' '} {row[2] if row[2] != 0 else ' '} |")
    print("-------")

# --- MAIN SCRIPT ---

# Start and goal states
start = [1, 2, 3, 4, 0, 5, 7, 8, 6]
goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]

solution = a_star_8_puzzle(start, goal)

if solution:
    # This is the line that was changed:
    print(f"Solved in {len(solution) - 1} moves! Here are the steps:")
    for i, step in enumerate(solution):
        print(f"\nStep {i}:")
        visualize_puzzle(step) # Use the new function
else:
    print("No solution found.")

Solved in 2 moves! Here are the steps:

Step 0:
-------
| 1 2 3 |
| 4   5 |
| 7 8 6 |
-------

Step 1:
-------
| 1 2 3 |
| 4 5   |
| 7 8 6 |
-------

Step 2:
-------
| 1 2 3 |
| 4 5 6 |
| 7 8   |
-------


In [ ]:
import heapq

# --- A* helpers ---
GOAL = (1,2,3,4,5,6,7,8,0)
GOAL_POS = {n: (i//3, i%3) for i, n in enumerate(GOAL)}  # target row,col for each tile

def h(state):
    # Manhattan distance of tiles to their goal positions (ignore 0)
    return sum(abs(r - GOAL_POS[n][0]) + abs(c - GOAL_POS[n][1])
               for i, n in enumerate(state) if n
               for r,c in [(i//3, i%3)])

def neighbors(state):
    s = list(state)
    z = s.index(0)
    r, c = divmod(z, 3)
    for dr, dc in ((-1,0),(1,0),(0,-1),(0,1)):
        nr, nc = r+dr, c+dc
        if 0 <= nr < 3 and 0 <= nc < 3:
            nz = 3*nr + nc
            t = s[:]
            t[z], t[nz] = t[nz], t[z]
            yield tuple(t)

def reconstruct(came_from, cur):
    path = [cur]
    while cur in came_from:
        cur = came_from[cur]
        path.append(cur)
    return path[::-1]

def astar(start):
    start = tuple(start)
    pq = [(h(start), 0, start)]
    came_from = {}
    g = {start: 0}
    seen = set()

    while pq:
        _, gc, cur = heapq.heappop(pq)
        if cur == GOAL:
            return reconstruct(came_from, cur)
        if cur in seen:
            continue
        seen.add(cur)

        for nb in neighbors(cur):
            ng = gc + 1
            if ng < g.get(nb, 1e9):
                g[nb] = ng
                came_from[nb] = cur
                heapq.heappush(pq, (ng + h(nb), ng, nb))
    return None

# --- demo ---
def show(state):
    print("-------")
    for i in range(0,9,3):
        row = [' ' if x==0 else x for x in state[i:i+3]]
        print(f"| {row[0]} {row[1]} {row[2]} |")
    print("-------")

start = (1,2,3,4,0,5,7,8,6)
sol = astar(start)
if sol:
    print(f"Solved in {len(sol)-1} moves")
    for i,s in enumerate(sol):
        print(f"Step {i}:")
        show(s)
else:
    print("No solution")
